<a href="https://colab.research.google.com/github/MardonioR/Proyecto-Polifenoles/blob/main/FE_Miriam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Incluye aquí todas las librerías que consideres necesarias:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import power_transform
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import LabelEncoder

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

DIR = "/content/drive/MyDrive/Colab Notebooks/"

os.chdir(DIR)

# df = pd.read_csv("Final_DF.csv", sep=",", encoding='latin1')
# df = pd.read_excel(r"Data/Final_DF.xlsx") # Changed path from ../Data to Data
# Use the direct download link for Google Sheets
gsheet_url = "https://docs.google.com/spreadsheets/d/1Z1Ubd2ZBnFw9B2NLIwyxZK69Pk4mNFRe/export?format=xlsx"
df = pd.read_excel(gsheet_url)
df.describe()

Mounted at /content/drive


,id,visita,grupo,periodo,tiempo,tratamiento,frec_alcohol_audit,puntaje_audit,Palabras_moca,memoria_moca,...,horas_cama,pgh_por_eficiencia,pgh_eficiencia,pgh_perturbaciones,pgh_disfuncion_dia,pgh_duracion_sueño,pgh_total,cat_pgh_total,total_meds,menst_flag
count,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,160.000000,160.000000,168.000000,168.000000,...,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000
mean,23.238095,2.500000,1.428571,1.500000,1.500000,1.500000,2.250000,3.200000,18.547619,3.880952,...,7.798413,85.257772,0.571429,1.208333,0.797619,1.250000,6.059524,1.422619,0.494048,0.488095
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,6.000000,1.000000,...,5.166667,33.333330,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
25%,12.000000,1.750000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,16.000000,3.000000,...,7.000000,80.000000,0.000000,1.000000,0.000000,1.000000,4.000000,1.000000,0.000000,0.000000
50%,22.500000,2.500000,1.000000,1.500000,1.500000,1.500000,2.000000,3.000000,18.500000,4.000000,...,7.500000,87.500000,0.000000,1.000000,1.000000,1.000000,5.000000,1.000000,0.000000,0.000000
75%,35.000000,3.250000,2.000000,2.000000,2.000000,2.000000,3.000000,4.000000,21.000000,5.000000,...,8.354166,93.333340,1.000000,1.000000,1.000000,2.000000,8.000000,2.000000,0.000000,1.000000
max,47.000000,4.000000,2.000000,2.000000,2.000000,2.000000,4.000000,12.000000,31.000000,5.000000,...,12.250000,104.000000,3.000000,2.000000,3.000000,3.000000,17.000000,2.000000,6.000000,1.000000
std,13.420530,1.121376,0.496351,0.501495,0.501495,0.501495,1.115922,2.256507,5.119891,0.983757,...,1.052518,11.857079,0.892896,0.487621,0.785799,0.880596,3.543912,0.495453,1.173546,0.501353


## **PCA**
Técnicamente: Realiza una transformación lineal ortogonal para proyectar los datos en un nuevo espacio donde la varianza se maximiza. Las nuevas características (PC1, PC2...) son combinaciones lineales de todas las variables originales.
Por qué no: El "Componente 1" podría ser 0.5*Edad - 0.2*Glucosa + 0.1*Peso. Clínicamente, esto es casi imposible de traducir en un perfil de paciente accionable.

In [ ]:
# Load the Excel file with variable types
# Replace 'nombre_del_archivo_tipos.xlsx' with the actual file name
# If the file is in a different directory, adjust the path accordingly (e.g., r'../OtherFolder/nombre_del_archivo_tipos.xlsx')
# Use the direct download link for Google Sheets
gsheet_tipos_url = "https://docs.google.com/spreadsheets/d/1_mDVnJONYaBi_ydvUiMAmKrbLq2eScGh/export?format=xlsx"
df_tipos_variables = pd.read_excel(gsheet_tipos_url)

# Display the first few rows and information about the new dataframe
print("Primeras filas del DataFrame de tipos de variables:")
display(df_tipos_variables.head())

print("Información general del DataFrame de tipos de variables:")
df_tipos_variables.info()

Primeras filas del DataFrame de tipos de variables:


,feature,type,metric,value,keep
0,tiempo_protrombina_rati,Numerica,zvar,1.0,True
1,wcst_ep_t,Numerica,zvar,1.0,True
2,cv_enviroment_100,Numerica,zvar,1.0,True
3,ffq_cafes,Numerica,zvar,1.0,True
4,ffq_bebida_energetica,Numerica,zvar,1.0,True


Información general del DataFrame de tipos de variables:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 518 entries, 0 to 517
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   feature  518 non-null    object 
 1   type     518 non-null    object 
 2   metric   518 non-null    object 
 3   value    518 non-null    float64
 4   keep     518 non-null    bool   
dtypes: bool(1), float64(1), object(3)
memory usage: 16.8+ KB


In [ ]:
# Identify categorical columns from df_tipos_variables that should be kept
categorical_features_to_encode = df_tipos_variables[
    (df_tipos_variables['type'] == 'Categorica') & (df_tipos_variables['keep'] == True)
]['feature'].tolist()

# Filter out features that are not present in df (in case of discrepancies)
categorical_features_to_encode = [col for col in categorical_features_to_encode if col in df.columns]

# Apply one-hot encoding to these selected categorical columns in df
df_encoded_features = pd.get_dummies(df[categorical_features_to_encode], drop_first=False)

# Drop the original categorical columns from df
df = df.drop(columns=categorical_features_to_encode)

# Concatenate the one-hot encoded features back to df
df = pd.concat([df, df_encoded_features], axis=1)

print("One-hot encoding applied to categorical variables based on df_tipos_variables.")
print(f"New shape of df: {df.shape}")
print("First 5 rows of the updated DataFrame:")
display(df.head())

One-hot encoding applied to categorical variables based on df_tipos_variables.
New shape of df: (168, 757)
First 5 rows of the updated DataFrame:


,id,visita,grupo,periodo,tiempo,tratamiento,frec_alcohol_audit,puntaje_audit,Palabras_moca,memoria_moca,...,tmb_1837,tmb_1842,tmb_1845,tmb_1847,tmb_1854,tmb_1950,tmb_1954,tmb_1967,tmb_2006,tmb_error báscula
0,1,1,1,1,1,1,2.0,5.0,29,4,...,False,False,False,False,False,False,False,False,False,False
1,1,2,1,1,2,1,2.0,5.0,29,4,...,False,False,False,False,False,False,False,False,False,False
2,1,3,1,2,1,2,2.0,5.0,29,4,...,False,False,False,False,False,False,False,False,False,False
3,1,4,1,2,2,2,2.0,5.0,29,4,...,False,False,False,False,False,False,False,False,False,False
4,2,1,2,1,1,2,2.0,2.0,21,4,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
print("Current shape of df:", df.shape)
display(df.head())

Current shape of df: (168, 757)


,id,visita,grupo,periodo,tiempo,tratamiento,frec_alcohol_audit,puntaje_audit,Palabras_moca,memoria_moca,...,tmb_1837,tmb_1842,tmb_1845,tmb_1847,tmb_1854,tmb_1950,tmb_1954,tmb_1967,tmb_2006,tmb_error báscula
0,1,1,1,1,1,1,2.0,5.0,29,4,...,False,False,False,False,False,False,False,False,False,False
1,1,2,1,1,2,1,2.0,5.0,29,4,...,False,False,False,False,False,False,False,False,False,False
2,1,3,1,2,1,2,2.0,5.0,29,4,...,False,False,False,False,False,False,False,False,False,False
3,1,4,1,2,2,2,2.0,5.0,29,4,...,False,False,False,False,False,False,False,False,False,False
4,2,1,2,1,1,2,2.0,2.0,21,4,...,False,False,False,False,False,False,False,False,False,False


# Task
Identify numerical features marked as 'keep' in `df_tipos_variables` that are of 'Numerica' type and present in `df`. Then, apply `MinMaxScaler` to these features in `df`, replace the original columns with the scaled versions, and display the head and shape of the updated `df`.

## Identify numerical features for scaling

### Subtask:
Filter `df_tipos_variables` to get a list of numerical features marked as 'keep' that are also present in `df`.


**Reasoning**:
To identify the numerical features that need to be scaled, I will filter the `df_tipos_variables` DataFrame based on 'Numerica' type and 'keep' status, and then ensure these features exist in the main `df` DataFrame.



In [ ]:
numerical_features_to_scale = df_tipos_variables[
    (df_tipos_variables['type'] == 'Numerica') & (df_tipos_variables['keep'] == True)
]['feature'].tolist()

# Filter out features that are not present in df (in case of discrepancies)
numerical_features_to_scale = [col for col in numerical_features_to_scale if col in df.columns]

print(f"Identified {len(numerical_features_to_scale)} numerical features for scaling.")
print("First 5 numerical features to scale:")
print(numerical_features_to_scale[:5])

Identified 370 numerical features for scaling.
First 5 numerical features to scale:
['tiempo_protrombina_rati', 'wcst_ep_t', 'cv_enviroment_100', 'ffq_cafes', 'ffq_bebida_energetica']


**Reasoning**:
Now that the numerical features have been identified, the next step is to apply `MinMaxScaler` to these features in the `df` DataFrame. This will scale the values of these columns to a specified range (defaulting to 0 and 1). After scaling, the original columns in `df` will be replaced with their scaled versions, and the head and shape of the updated DataFrame will be displayed to verify the changes.



In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Apply MinMaxScaler to the identified numerical features
df[numerical_features_to_scale] = scaler.fit_transform(df[numerical_features_to_scale])

print("MinMaxScaler applied to numerical features.")
print(f"Updated shape of df: {df.shape}")
print("First 5 rows of the updated DataFrame with scaled numerical features:")
display(df.head())

MinMaxScaler applied to numerical features.
Updated shape of df: (168, 757)
First 5 rows of the updated DataFrame with scaled numerical features:


,id,visita,grupo,periodo,tiempo,tratamiento,frec_alcohol_audit,puntaje_audit,Palabras_moca,memoria_moca,...,tmb_1837,tmb_1842,tmb_1845,tmb_1847,tmb_1854,tmb_1950,tmb_1954,tmb_1967,tmb_2006,tmb_error báscula
0,1,1,1,1,1,1,2.0,5.0,0.92,4,...,False,False,False,False,False,False,False,False,False,False
1,1,2,1,1,2,1,2.0,5.0,0.92,4,...,False,False,False,False,False,False,False,False,False,False
2,1,3,1,2,1,2,2.0,5.0,0.92,4,...,False,False,False,False,False,False,False,False,False,False
3,1,4,1,2,2,2,2.0,5.0,0.92,4,...,False,False,False,False,False,False,False,False,False,False
4,2,1,2,1,1,2,2.0,2.0,0.60,4,...,False,False,False,False,False,False,False,False,False,False


## Summary:

### Data Analysis Key Findings

*   A total of 370 numerical features were identified for scaling. These features met the criteria of being of type 'Numerica', marked as 'keep' in `df_tipos_variables`, and present in the `df` DataFrame.
*   The first five identified features included `['tiempo_protrombina_rati', 'wcst_ep_t', 'cv_enviroment_100', 'ffq_cafes', 'ffq_bebida_energetica']`.
*   The `MinMaxScaler` was successfully applied to these 370 numerical features within the `df` DataFrame.
*   After scaling, the `df` DataFrame maintained its original shape of (168, 757), confirming that the scaling was performed in-place.
*   The head of the updated DataFrame showed that the numerical feature values were transformed, indicating the successful application of `MinMaxScaler`, typically resulting in values between 0 and 1.

### Insights or Next Steps

*   The data is now prepared with scaled numerical features, which is crucial for many machine learning algorithms that are sensitive to feature scales, such as gradient descent-based optimizers or distance-based algorithms.
*   The `df` DataFrame is now ready for subsequent steps in the data pipeline, which may include model training, dimensionality reduction, or further exploratory analysis.
